# Mask Refinement for semantic segmentation training

This notebook is thought for all those times in which you have to train a network for image segmentation purposes but the masks provided or made by hand are not accurate enough (for instance, semantic segmentation of leaves).

### Methodology

The idea is very simple: apply gaussian mixtures to the mask only. However, for certain situations it proved to be better to work in the whole image and then update the old mask with the new values just in the "ones" of the original mask.


In [ ]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture 



In [ ]:
#uploading mask and img
img= cv2.imread('img.jpg') 
mask= cv2.imread('mask.jpg',0) #black and white

mask[mask>=(255/2)]=255
mask[mask<(255/2)]=0

In [ ]:
# we need just the mask "ones" in the image
mask_in_img=img[mask==255]
mask_in_img.shape

In [ ]:
'''
you can use one model for all the masks, or generate a model for each mask. This second option takes more time,
however sometimes the distributions are so different and it's better to have one for each mask.
'''

gm_local = GaussianMixture(n_components=2,n_init=5, covariance_type='full',random_state=1).fit(mask_in_img) 
'''
params:
-for a binary mask usualy you work with 2 components
-n_init is the time you run the model, it keeps the best
-covariance_type can be full, tied, spherical etc
- random state to keep the same results along different instances
'''
print(gm_local.converged_,"converged")
print(gm_local.n_iter_,"n iteration to converge")
prediction = gm_local.predict(mask_in_img)

In [ ]:
#set new values in the mask
refined_mask=np.copy(mask)
refined_mask[refined_mask==255]=prediction*255

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].imshow(img)
ax[1].imshow(refined_mask)

In [ ]:
# save the mask
refined_mask_path='your_path/'

from skimage import io

io.imsave(refined_mask_path,refined_mask)